# Graph and networks Lab 2

## Place your imports here

In [1]:
import base64
import requests

from dns import message
from pprint import pprint
from ipaddress import IPv4Address, AddressValueError

## PRIVATE RIPE Atlas key (provided during the session).

In [ ]:
# DO NOT SHARE OR EXCHANGE THE PROVIDED API KEY #
API_KEY = ""

## Default Parameters

In [3]:
HEADER = {"Authorization": f"Key {API_KEY}"}
RIPE_ATLAS_API_URL = "https://atlas.ripe.net/api/v2"

## DNS measurements

* For both VPs [7193, 6830] perform DNS resolution to:
    * google.com
    * cloudflare.com
* Function get_dns_results(m_id) parse and return the result of a DNS measurement

In [4]:
VP_IDS = [7193, 6830]
HOSTNAME1 = "www.cloudflare.com"
HOSTNAME2 = "www.google.com"

In [5]:
def get_dns_config(
    target: str,
    vp_ids: list[int],
    description: str = "GaN DNS",
    resolver_address: str = None,
) -> dict:
    """function to define a DNS measurement before posting it"""

    params = {
        "type": "dns",
        "af": 4,
        "query_class": "IN",
        "query_type": "A",
        "query_argument": target,
        "tags": ["dioptra", "gan"],
        "description": f"{description}",
        "is_oneoff": True,
    }

    if resolver_address:
        params["resolver_address"] = resolver_address
    else:
        params["use_probe_resolver"] = True

    return {
        "definitions": [params],
        "probes": [
            {"value": v_id, "type": "probes", "requested": 1} for v_id in vp_ids
        ],
        "is_oneoff": True,
    }

In [ ]:
url = f"{RIPE_ATLAS_API_URL}/measurements/"

m_id_1 = requests.post(url=url, headers=HEADER, json=get_dns_config(HOSTNAME1, VP_IDS))
m_id_1 = m_id_1.json()["measurements"][0]
print(m_id_1)

In [ ]:
# measurement results url
m_id_1 = 129746259
url = f"{RIPE_ATLAS_API_URL}/measurements/{m_id_1}/results"
print(url)

resp = requests.get(url)
resp = resp.json()

In [8]:
def parse_dns_results(resp: list[dict]) -> list[dict]:
    """return DNS results based for a measurement"""

    parsed_results = []
    for m in resp:
        src_addr = m["from"]
        prb_id = m["prb_id"]
        results = m["resultset"]

        for result in results:
            resolver_addr = result["dst_addr"]
            dns_results = result["result"]
            abuf = dns_results["abuf"]
            try:
                msg = message.from_wire(base64.b64decode(abuf))
            except Exception as e:
                print(e)
                continue

            for ans in msg.answer:
                hostname = ans.name.to_text()
                ipv4_redirection = set()
                for item in ans.items:
                    try:
                        if IPv4Address(item).is_private:
                            continue
                    except AddressValueError:
                        continue

                    ipv4_redirection.add(str(item))
                parsed_results.append(
                    {
                        "src_addr": src_addr,
                        "prb_id": prb_id,
                        "resolver_addr": resolver_addr,
                        "hostname": hostname,
                        "answers": ipv4_redirection,
                    }
                )

    return parsed_results

In [ ]:
dns_results_1 = parse_dns_results(resp)
for result in dns_results_1:
    src_addr = result["src_addr"]
    prb_id = result["prb_id"]
    resolver_addr = result["resolver_addr"]
    hostname = result["hostname"]
    answers = result["answers"]
    print(f"{src_addr=}; {prb_id=}; {resolver_addr=}; {hostname=}; {answers=}")

## Write the code for measuring the second hostname

# Perform Traceroute

## Import dependencies

In [1]:
from collections import defaultdict
from helpers import get_prefix_from_ip
from ripe_atlas_api import get_traceroute_config

# defaultdict is a very helpful python data structure that facilitate dict manipulation
# example: a dict of dict or a dict of list
my_default_dict = defaultdict(list)
for i in range(0, 10, 1):
    if i < 5:
        my_default_dict["a"].append(i)
    else:
        my_default_dict["b"].append(i)
print(my_default_dict)

defaultdict(<class 'list'>, {'a': [0, 1, 2, 3, 4], 'b': [5, 6, 7, 8, 9]})


* You retrieved one or more IP addresses per pair (VP; hostname):
    1. For each IP address, store the pair (VP; hostname)
    2. Retrieved one IP address per /24 prefix for each pair
    3. Perform meshed Traceroute for each IP addresses (between 10-15 measurements)
    4. Retrieve results and group them per (VP; hostname; dst_addr)

In [ ]:
# EXERCISE 1: Get one IP address per /24 prefix answers
# You can use the helper function which return the /24 prefix by default
# ex: ip_addr = 8.8.8.8, get_prefix_from_ip(ip_addr, 24) returns ip_prefix =  8.8.8.0/24
# You should obtain a list of targets to trace

In [ ]:
# EXERCISE 2: Run Traceroute measurements and retrieve measurements
# HINT: For each measurements, get the measurement id and store in a list

# Print latency for each pair VP; hostname

# For each traceroute, get the AS Path using RIPE RIS

## Query RIPE RIS API

In [ ]:
ip_addr = "132.227.123.74"
url = "https://stat.ripe.net/data/prefix-overview/data.json"
params = {"resource": ip_addr}
resp = requests.get(url, params=params)
resp = resp.json()
pprint(resp)

{'build_version': 'main-2025.09.19',
 'cached': True,
 'data': {'actual_num_related': 0,
          'announced': True,
          'asns': [{'asn': 1307, 'holder': 'FR-U-JUSSIEU-PARIS'}],
          'block': {'desc': 'Administered by ARIN',
                    'name': 'IANA IPv4 Address Space Registry',
                    'resource': '132.0.0.0/8'},
          'is_less_specific': True,
          'num_filtered_out': 0,
          'query_time': '2025-09-23T08:00:00',
          'related_prefixes': [],
          'resource': '132.227.0.0/16',
          'type': 'prefix'},
 'data_call_name': 'prefix-overview',
 'data_call_status': 'supported',
 'messages': [['warning',
               'Given resource is not announced but result has been aligned to '
               'first-level less-specific (132.227.0.0/16).']],
 'process_time': 1,
 'query_id': '20250923122459-e29779ea-25d4-4037-b5c0-1152e15be7a6',
 'see_also': [],
 'server_id': 'app179',
 'status': 'ok',
 'status_code': 200,
 'time': '2025-09-23T1

In [ ]:
# EXERCISE: get the AS number and BGP prefix

## Output AS path for each of your Traceroutes